# < Meaningful title >

##### Imports

In [1]:
import requests
import googlemaps
import pandas as pd
import numpy as np
#from geopy.distance import geodesic
import json
#import folium
import time
from datetime import datetime
from itertools import permutations
#from geopy.distance import great_circle
from io import StringIO

# Custom util functions
import sys; sys.path.append("./libraries/")
from utils import *

### Settings

##### Reproducibility settings

In [2]:
# Random seed
np.random.seed = 7

# Relative Paths
raw_data = "../data/raw_data/"
process_data = "../data/process_data"

# Flags
collect = False # Flag to collect data or load existent raw_data

##### Google API

In [3]:
key = open("./Google_API_key.txt").readline()
gmaps = googlemaps.Client(key=key)

# 1. Data Collection

We start by creating a list of query values that relate to the dataset. We are interested in getting mostly reviews (and some other metadata) on specific fitness facilities (i.e. popular chains) from main cities in Denmark. To do this, we will compute the query list as a combination of cities and fitness chains. 

In [4]:
# List of cities
cities = ['Copenhagen', 'Aalborg', 'Arhus', 'Odense']
 
# Popular fitness chains
gyms = ["PureGym", "SATS", "Vesterbronx"]

# Query list
query_list = [g + " " + c for g in gyms for c in cities]

print(query_list)

['PureGym Copenhagen', 'PureGym Aalborg', 'PureGym Arhus', 'PureGym Odense', 'SATS Copenhagen', 'SATS Aalborg', 'SATS Arhus', 'SATS Odense', 'Vesterbronx Copenhagen', 'Vesterbronx Aalborg', 'Vesterbronx Arhus', 'Vesterbronx Odense']


## 1.1 Google Maps API

The Google maps API takes a single query string to search for results (similar to the User Interface searchbox). Therefore, we combine popular fitness facilities with main Danish cities as our query keys.

### 1.1.1 Reviews
We start by getting the reviews for our query list.

Get responses for all the queries from the API

In [5]:
if collect:
    # Get response for queries
    dfs = []

    # For each query in the query list
    for query in query_list:  
        # Get the response using our custom made querier
        dfs.append(google_querier(gmaps, query))

    google_reviews = pd.concat(dfs)

    # Save to disk
    google_reviews.to_csv(raw_data + "google_reviews.csv", index=False, encoding="utf-8")

else:
    google_reviews = pd.read_csv(raw_data + "google_reviews.csv")

Check the results.

In [6]:
check_dataframe_results(google_reviews)

Resulting dataframe has shape (360, 9)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       360 non-null    object 
 1   type           360 non-null    object 
 2   name           360 non-null    object 
 3   lat            360 non-null    float64
 4   lng            360 non-null    float64
 5   author_name    360 non-null    object 
 6   rating         360 non-null    int64  
 7   text           360 non-null    object 
 8   opening_hours  360 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 25.4+ KB
None


,place_id,type,name,lat,lng,author_name,rating,text,opening_hours
0,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,PureGym,55.669812,12.54739,madi sharp,4,"Sweet small gym, staff are kind when you see t...","{'Monday': '05:00AM - 12:00AM', 'Tuesday': '05..."
1,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,PureGym,55.669812,12.54739,Lewis Atkins,2,"Just a very bad gym. Staff don’t really care, ...","{'Monday': '05:00AM - 12:00AM', 'Tuesday': '05..."
2,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,PureGym,55.669812,12.54739,Eric,1,"terrible facilities\nbathrooms are gross, dirt...","{'Monday': '05:00AM - 12:00AM', 'Tuesday': '05..."
3,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,PureGym,55.669812,12.54739,Rune Perstrup,1,An Unhygienic Coronavirus Petri Dish.\n\nI hav...,"{'Monday': '05:00AM - 12:00AM', 'Tuesday': '05..."
4,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,PureGym,55.669812,12.54739,Mario Piazza,1,In a huge gym there is only one hair dryer and...,"{'Monday': '05:00AM - 12:00AM', 'Tuesday': '05..."


### 1.1.2 Nearby Transportation
We are interested in collecting the nearby transportation to the fitness centers.

In [7]:
if collect:

    # Radius of search in meters
    radius = 500

    # Transportation type key (similar to what one would input in Google Maps search box)
    transportation_type = ['bus_station', 'train_station', 'transit_station'] # Avaliable transportation: only bus station, train station and transit station (which includes metro)

    # Container
    nearby_transportation = []

    # We iterate through all our fitness centers, and retrieve nearby transportations
    for ix, row in google_reviews.iterrows():
        # Extract info from fitness center
        place_id = row.place_id
        location = {"lat": row.lat, "lng": row.lng}
        # Look at nearby transportation
        df = google_nearby(gmaps, place_id = place_id, keys = transportation_type, location = location, radius = radius)
        # Append results
        nearby_transportation.append(df)

    # Join all results
    nearby_transportation = pd.concat(nearby_transportation)

    # Save to disk
    nearby_transportation.to_csv(raw_data + "transportation.csv", index=False, encoding="utf-8")

else:
    nearby_transportation = pd.read_csv(raw_data + "transportation.csv")

Check the results.

In [8]:
check_dataframe_results(nearby_transportation)

Resulting dataframe has shape (6195, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6195 entries, 0 to 6194
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   place_id                6195 non-null   object 
 1   transport_id            6195 non-null   object 
 2   transport_name          6195 non-null   object 
 3   transport_type          6195 non-null   object 
 4   transport_lat           6195 non-null   float64
 5   transport_lng           6195 non-null   float64
 6   distance_gym_transport  6195 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 338.9+ KB
None


,place_id,transport_id,transport_name,transport_type,transport_lat,transport_lng,distance_gym_transport
0,ChIJh3mB6UxSUkYREbiH4JDK-7M,ChIJ-y92w3VTUkYRY8NOJNuwQkQ,Kridt v/Rikke Frisk,bus_station,55.668036,12.551084,305
1,ChIJh3mB6UxSUkYREbiH4JDK-7M,ChIJ76k9hJ9TUkYRRaHYcLRX3XE,Lysholdet v/Jakob Holst,bus_station,55.672786,12.547279,331
2,ChIJh3mB6UxSUkYREbiH4JDK-7M,ChIJydMagp9TUkYRjfgBz2sKErQ,"Ejerforeningen Sigbrits Allé 3, 5 og 5a",bus_station,55.672865,12.546713,343
3,ChIJh3mB6UxSUkYREbiH4JDK-7M,ChIJydMagp9TUkYRcgSv8LkczNs,Grundejerforeningen Carl Jacobsens Vej 33-41,bus_station,55.672865,12.546713,343
4,ChIJh3mB6UxSUkYREbiH4JDK-7M,ChIJl46JnZ5TUkYRnW93t25gFhQ,Optiperform v/Maja Juel-Hansen,bus_station,55.668011,12.542917,346


## 1.2 Trustpilot WebCrawler

Trustpilot is a Danish consumer review website very popular in Denmark. It is publicly available and easy to access, but it does not provide any API integration. Therefore, we use a simple webcrawler to extract the reviews of interest.

In [9]:
if collect:
    dfs = []

    # Reuse the gyms
    for g in gyms:
        df = trustpilot_crawler(key=g, verbose=False)

        # Append the facility DF to main df
        dfs.append(df)

    # Join all DFs
    trustpilot_reviews = pd.concat(dfs)

    # Save to disk
    trustpilot_reviews.to_csv(raw_data + "trustpilot_reviews.csv", index=False, encoding="utf-8")

else:
    trustpilot_reviews = pd.read_csv(raw_data + "trustpilot_reviews.csv")

Check the results.

In [10]:
check_dataframe_results(trustpilot_reviews)

Resulting dataframe has shape (2802, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2802 entries, 0 to 2801
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   datetime    2802 non-null   object
 1   name        2802 non-null   object
 2   rating      2802 non-null   int64 
 3   title       2802 non-null   object
 4   review      2802 non-null   object
 5   event_time  2802 non-null   object
 6   enterprise  2802 non-null   object
dtypes: int64(1), object(6)
memory usage: 153.4+ KB
None


,datetime,name,rating,title,review,event_time,enterprise
0,2023-11-13T14:03:40.000Z,Jan Winther,4,Godt fitness-center,Gennemgående er jeg godt tilfreds med mit fitn...,13. november 2023,PureGym
1,2023-11-14T13:07:20.000Z,Tina Holst,5,Syntes altid det er dejligt at komme i…,Syntes altid det er dejligt at komme i centret...,14. november 2023,PureGym
2,2023-11-13T09:22:36.000Z,Pfændtner,5,Jeg har gået i Fitness centeret i 22år…,Jeg har gået i Fitness centeret i 22år og efte...,12. november 2023,PureGym
3,2023-11-13T17:18:33.000Z,Gitte,5,Puregym Ikast,Puregym Ikast er et fantastisk center. Man føl...,13. november 2023,PureGym
4,2023-11-13T10:01:35.000Z,GITTE MIKKELSEN,2,Der mangler Stram op hold,Der mangler Stram op hold (eller ligende fx Pu...,11. november 2023,PureGym


## 1.3 Københavns Kommune WebCrawler

The Københavns Kommune website provides an extensive list of training facilities, both indoors and outdoors. Since this is a dynamic site built on JavaScript, the traditional webcrawler approach is not suitable, and thus we will use an approach that simulates human-like interactions using Selenium.

In [11]:
if collect:
    # Target URL
    url = 'https://kbhkort.kk.dk/spatialmap?page=widget-view&name=motion/motionslisten'

    # Data container
    data = {'type':[], 'website':[], 'location':[], 'address':[]}

    # Options for ChromeDriver
    options = ChromeOptions()       # Get Chrome options
    options.headless = True         # This stops an actual browser from being open and shown
    # Create instance
    driver = Chrome(r"C:\Users\chris\OneDrive\MSc\KDS_A2023_Web\chromedriver-win64\chromedriver.exe", options=options)  # Optional argument, if not specified will search path.
    driver.get(url)

    print("Driver and URL passed. Wait a second...")
    driver.implicitly_wait(20) # Let the user actually see something!

    count = len(driver.find_elements(By.XPATH, '/html/body/div/div[4]/div/ul/li'))
    print(f'There is a total of {count} entries.')

    for i in tqdm(range(1, count+1)): # count+1
        # Type of activity
        type = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[1]/strong[1]')
        data['type'].append(type.text)

        # Location of activity - not all entries list a location
        try:
            location = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[1]/strong[2]')
            data['location'].append(location.text)
        except NoSuchElementException:
            data['location'].append(None)
        # Website
        try:
            site = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[2]/span[1]/a')
            data['website'].append(site.get_attribute('href'))
        except NoSuchElementException:
            data['website'].append(None)

        # Address of activity - some entries have an extra field, so this messess with the current (janky) XPath implementation
        try:
            address = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[2]/span[6]')
            if not address.text.startswith('Mødested: '): # Check that the text is indeed the address
                try:
                    address = driver.find_element(By.XPATH, f'/html/body/div/div[4]/div/ul/li[{i}]/div[2]/div[2]/span[7]')
                    data['address'].append((address.text).removeprefix('Mødested: '))
                except NoSuchElementException:
                    data['address'].append(None)
            else: 
                data['address'].append((address.text).removeprefix('Mødested: '))
        except NoSuchElementException:
            data['address'].append(None)

    driver.quit()

    kbh_facilities = pd.DataFrame.from_dict(data)

    # Save to disk
    kbh_facilities.to_csv(raw_data + "kbh_facilities.csv", index=False, encoding="utf-16") # Since some Danish characters don't map to utf-8, we use utf-16
    

else:
    kbh_facilities = pd.read_csv(raw_data + "kbh_facilities.csv", encoding="utf-16")

Check the results.

In [12]:
check_dataframe_results(kbh_facilities)

Resulting dataframe has shape (610, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      610 non-null    object
 1   website   526 non-null    object
 2   location  593 non-null    object
 3   address   526 non-null    object
dtypes: object(4)
memory usage: 19.2+ KB
None


,type,website,location,address
0,Styrke- og grundtræning,http://www.sosmotion.dk/,SOS Motion,"Sundhedshus Østerbro, Randersgade 60, 4 sal, 2..."
1,Træningspavillion,NaN,NaN,"Kvægtorvsgade, 1710 KBH V"
2,Kondisti,NaN,Valbyparken,"Tudsemindevej, 2450 Valby"
3,Nærgymnastik,https://lofskolen.dk/kurser/motion-og-sundhed/...,LOFskolen,"Østerbrogade 240, 2100 København Ø"
4,Stavgang for seniorer,https://kbhkg.klub-modul.dk/default.aspx,Københavns Gymnastikforening,"Ole Suhrs Gade 3, 1354 København"


We observe that this dataset only contains addresses, but not geolocation (latitude and longitude). We then try to collect that data from Google Maps.

In [20]:
if collect:
    # We create the new columns
    kbh_facilities['lat'] = None
    kbh_facilities['lng'] = None

    # Subset only facilities with either address OR location
    for index, row in kbh_facilities[~kbh_facilities.isna()].iterrows():

        #look first if we can find lat and lng for the address
        lat_lng = get_lat_lng(gmaps, row.address)
        
        # if there is no address or we can't find the coordiantes using address we try the location
        if not lat_lng:
            lat_lng = get_lat_lng(gmaps, row.location)
            
        # if none of location and address works we continue with the next (leave it at None)
        if not lat_lng:
            continue

        kbh_facilities.at[index, 'lat'] = lat_lng[0]
        kbh_facilities.at[index, 'lng'] = lat_lng[1]

    # Save to disk
    kbh_facilities.to_csv(raw_data + "kbh_facilities.csv", index=False, encoding="utf-16") # Since some Danish characters don't map to utf-8, we use utf-16

else:
    kbh_facilities = pd.read_csv(raw_data + "kbh_facilities.csv", encoding="utf-16")

Check the results.

In [21]:
check_dataframe_results(kbh_facilities)

Resulting dataframe has shape (610, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   type      610 non-null    object 
 1   website   526 non-null    object 
 2   location  593 non-null    object 
 3   address   526 non-null    object 
 4   lat       518 non-null    float64
 5   lng       518 non-null    float64
dtypes: float64(2), object(4)
memory usage: 28.7+ KB
None


,type,website,location,address,lat,lng
0,Styrke- og grundtræning,http://www.sosmotion.dk/,SOS Motion,"Sundhedshus Østerbro, Randersgade 60, 4 sal, 2...",55.707451,12.580255
1,Træningspavillion,NaN,NaN,"Kvægtorvsgade, 1710 KBH V",55.669719,12.563130
2,Kondisti,NaN,Valbyparken,"Tudsemindevej, 2450 Valby",55.643635,12.525831
3,Nærgymnastik,https://lofskolen.dk/kurser/motion-og-sundhed/...,LOFskolen,"Østerbrogade 240, 2100 København Ø",55.713762,12.578719
4,Stavgang for seniorer,https://kbhkg.klub-modul.dk/default.aspx,Københavns Gymnastikforening,"Ole Suhrs Gade 3, 1354 København",55.688828,12.572720
